# Introduction

Dans ce TP, nous nous intéressons à une compagnie spatiale s'occupe de livraison. Chaque planète est gérée par un **gestionnaire de planètes** qui assigne des livraisons depuis la planète dont il est responsable vers une autre planète. Ce TP se focalisera principalement sur les aspects environnement, interaction et organisation du système multiagents.

## Environnement
Dans ce TP, l'environnement a deux composantes: la première concerne l'espace. Celui-ci est peuplé d'objets immobiles, les planètes, qui sont représentées par les gestionnaires qui les contrôlent. La seconde concerne les biens à livrer, qui sont des objets qui seront manipulés par les vaisseaux. Avant de fournir le code de la manière dont les biens sont codés, voici les imports nécessaires au TP:

In [1]:
import json  # Pour la sérialisation/désérialisation des objects
import math
import random
import string
from collections import defaultdict
from typing import List

import mesa
import mesa.space
import numpy as np
import spade  # Framework multi-agents de messages
import networkx as nx  # Pour le parcours du réseau de planètes
from mesa import Agent, Model
from threading import Lock  # Pour le mutual exclusion

from mesa.datacollection import DataCollector
from mesa.time import RandomActivation
from mesa.visualization import ModularVisualization
from mesa.visualization.ModularVisualization import VisualizationElement, ModularServer
from mesa.visualization.modules import ChartModule
from spade.behaviour import PeriodicBehaviour, OneShotBehaviour
from spade.template import Template
import uuid  # Génération de Unique ID

NEW_ITEM_PROBA = 0.05
PROBA_ISSUE_ROAD = 0.05
ROAD_BRANCHING_FACTOR = 0.5
WAITING_TIME = 3

In [2]:
class Item:
    @staticmethod
    def from_json(json_object):  #Désérialisation des items pour le passage par message
        return Item(json_object['x'], json_object['y'], json_object['a'], json_object['b'], json_object['c'],
                    json_object['uid'])

    def __init__(self, x, y, a=None, b=None, c=None, uid=None):
        if not a:
            self.a = random.random()
        else:
            self.a = a
        if not b:
            self.b = random.random()
        else:
            self.b = b
        if not c:
            self.c = random.random()
        else:
            self.c = c
        self.x = x
        self.y = y
        if not uid:
            self.uid = int(uuid.uuid1())
        else:
            self.uid = uid

    def __eq__(self, other):
        return isinstance(other, Item) and self.uid == other.uid

    def __hash__(self):
        return int(self.uid)

    @staticmethod
    def portrayal_method():
        color = "yellow"
        r = 2
        portrayal = {"Shape": "circle",
                     "Filled": "true",
                     "Layer": 3,
                     "Color": color,
                     "r": r}
        return portrayal

**Question 1 – Caractérisez l'environnement du système comme nous l'avons vu en cours. Justifiez.**

**Question 2 – La classe `Item` n'hérite pas de la classe `Agent` de mesa. En quoi cette entité ne constitue-t-elle pas un agent ?**

*Insérez votre réponse ici*

Un autre élément de l'environnement est le réseau de routes utilisé par les agents, `SpaceRoadNetwork`. Ce dernier est modélisé par un agent pour des raisons pratiques, mais il n'est ni autonome ni réactif. Le réseau de route constitue un graphe dont les nœuds sont les planètes et les arêtes sont les routes entre les planètes; ce sont ces routes qui sont empruntées par les agents. Ce graphe est modélisé au moyen du package `networkx`.

In [3]:
class SpaceRoadNetwork(Agent):
    def __init__(self, planets: List, unique_id: int, model: Model):
        super().__init__(unique_id, model)
        self.initial_graph = nx.Graph()
        self.current_graph = nx.Graph()
        self.speed_modificator = dict()
        for i in range(len(planets)):
            self.initial_graph.add_node(planets[i])
            for j in range(i):
                if random.random() < ROAD_BRANCHING_FACTOR:
                    distance = np.linalg.norm([planets[i].x - planets[j].x, planets[i].y - planets[j].y])
                    self.initial_graph.add_edge(planets[i], planets[j], distance=distance)
        # Reconnect graph of the roads between planets
        while len(list(nx.connected_components(self.initial_graph))) != 1:
            first_element = random.choice(tuple(list(nx.connected_components(self.initial_graph))[0]))
            second_element = random.choice(tuple(list(nx.connected_components(self.initial_graph))[1]))
            distance = np.linalg.norm([first_element.x - second_element.x, first_element.y - second_element.y])
            self.initial_graph.add_edge(first_element, second_element, distance=distance)
        for e in self.initial_graph.edges:
            self.current_graph.add_edge(e[0], e[1],
                                        distance=nx.get_edge_attributes(self.initial_graph, 'distance')[(e[0], e[1])])
            self.speed_modificator[e] = 1.0
            self.speed_modificator[(e[1], e[0])] = 1.0

    def step(self):
        pass

    def portrayal_method(self):
        portrayals = []
        for e in [edge for edge in self.current_graph.edges]:
            color = "green"
            portrayal = {"Shape": "line",
                         "width": 1,
                         "Layer": 1,
                         "Color": color,
                         "from_x": (tuple(e)[0].x - self.model.space.x_min) /
                                   (self.model.space.x_max - self.model.space.x_min),
                         "from_y": (tuple(e)[0].y - self.model.space.y_min) /
                                   (self.model.space.y_max - self.model.space.y_min),
                         "to_x": (tuple(e)[1].x - self.model.space.x_min) /
                                 (self.model.space.x_max - self.model.space.x_min),
                         "to_y": (tuple(e)[1].y - self.model.space.y_min) /
                                 (self.model.space.y_max - self.model.space.y_min)
                         }
            portrayals.append(portrayal)
        return portrayals

## Communication: introduction à spade

Il existe donc deux principaux types d'agents dans ce TP: les agents `PlanetManager` génèrent des biens et utilisent le **Contract Net Protocol** pour les proposer à la livraison. Les vaisseaux `Ship` essaient d'obtenir les biens qui leurs correspondent le mieux. Ces agents ont chacun besoin de connaître l'ensemble des agents de l'autre type. Les `PlanetManager` sont quant à eux obligés de connaître les autres `PlanetManager` afin d'en déterminer un qui soit le destinataire du bien. Dans tous les cas, ces agents sont des agents capables de communiquer. Pour représenter cela, nous allons utiliser un autre framework multi-agents, le framework [spade](https://spade-mas.readthedocs.io/) (*smart python agent developing framework*). Nous utiliserons ce frameworks dans les TP 4 et 5. Pour installer sapde, ouvrez un terminal et tapez:
```bash
pip install spade
```
Puisque nous utliserons aussi networkx, installez-le également en tapant:
```bash
pip install networkx
```

Contrairement à mesa, qui est conçu pour la simulation et le test d'hypothèses de recherches, spade est conçu spécifiquement pour faire de l'échange de message dans des environnements où les agents sont mobiles, peuvent se connecter depuis plusieurs machines etc. Les messages sont formatés en utilisant le protocole [xmpp](https://en.wikipedia.org/wiki/XMPP). Cependant, spade n'implémente pas lui-même un serveur xmpp, uniquement des clients pour les agents. Il va donc nous falloir installer un serveur xmpp. Nous utiliserons *prosody*, qui est conseillé dans la doucumentation de spade. Pour installer prosody, rendez-vous sur la page de prosody et **suivez scrupuleusement les instructions**. Et surtout **Ne restez pas bloqués**. Si vous n'arrivez pas à installer prosody, envoyez-moi un message. Une fois prosody installé, lancez un terminal et tapez:
```bash
sudo prosodyctl start
for i in {0..50};
    do sudo prosodyctl register ship-$i localhost password-ship-$i;
done
for i in {0..50};
    do sudo prosodyctl register planet-$i localhost password-planet-$i;
done
```
si vous êtes sous unix et
```powershell
prosodyctl start
for ($i=0; $i -le 50; $i++) {
    prosodyctl register ship-$i localhost password-ship-$i
}
for ($i=0; $i -le 50; $i++) {
    prosodyctl register ship-$i localhost password-planet-$i
}
```
si vous êtes sous windows powershell.

Une fois cela fait, nous avons donc un serveur qui tourne, et tous les agents que nous utiliserons inscrits. Pour cela, nous allons définir un agent spade de base capable de récupérer des messages et d'en envoyer. Cet agent sera utilisé comme un composant de l'agent mesa.

In [4]:
class AgentCommunicator(spade.agent.Agent):
    def __init__(self, jid, password):
        super().__init__(jid, password)
        self.msg_box = []
        self.mutex = Lock()
        self.send_behaviour = None

    class SendBehaviour(OneShotBehaviour):
        def __init__(self, msg):
            super().__init__()
            self.msg = msg

        async def run(self):
            await self.send(self.msg)
            print("sent: " + str(self.msg) + '\n')

    class RecvBehav(PeriodicBehaviour):
        async def run(self):
            msg = await self.receive()
            if msg:
                self.agent.mutex.acquire()
                self.agent.msg_box.append(msg)
                print("received: " + str(msg))
                self.agent.mutex.release()

    async def setup(self):
        b = self.RecvBehav(.01)
        self.add_behaviour(b, Template())
        print(str(self.jid) + " connected")


Notez que les actions des agents sont représentés par des `Behaviour`, qui peuvent être au moins `OneShot` (c'est à dire lancés une unique fois) ou `Periodic` c'est à dire lancés à intervalles réguliers (ici 0.01s). Lorsque l'agent est lancé, on ajoute aux comportements de l'agent son `PeriodicBehaviour` de réception. Ce dernier va recevoir les messages et les mettre dans la liste de messages correspondant à la boîte aux lettres de l'agent. Pour éviter les accès concurrents, on utilisera un **mutex** ou [mutual exclusion](https://en.wikipedia.org/wiki/Lock_(computer_science)).

Les agents capables de communiquer auront donc en commun de pouvoir envoyer des messages et d'avoir un "composant" qui est en fait un agent spade. On construit donc une classe qui correspond à ces agents, `CommunicatingAgent`:

In [5]:
class CommunicatingAgent(Agent):
    def __init__(self, unique_id: int, model: Model, name: string):
        super().__init__(unique_id, model)
        self.communicator = AgentCommunicator(name + "@localhost", "password-" + name)
        self.communicator.start()

    def send(self, msg):
        self.communicator.send_behaviour = AgentCommunicator.SendBehaviour(msg)
        self.communicator.add_behaviour(self.communicator.send_behaviour)
        self.communicator.send_behaviour.join()

On peut alors définir les agents capables de communiquer, le `PlanetManager`. Cet agent génère à temps régulier un nouvel `Item` qu'il faut livrer à une autre planète. Il génère alors un *call for proposal* pour transporter cet item, qu'il transmet à tous les `Ship` se trouvant sur sa planète.

In [6]:
class PlanetManager(CommunicatingAgent):
    def __init__(self, name: string, ships: List, unique_id: int, model, x, y):
        super().__init__(unique_id, model, name)
        self.x = x
        self.y = y
        self.items_to_ship = {}
        self.waiting_for_proposal = []
        self.ships = ships
        self.start_times = dict()
        self.proposals = dict()
        self.planets = []

    def step(self):
        if random.random() < NEW_ITEM_PROBA:
            item = Item(self.x, self.y)
            self.model.items.append(item)
            self.items_to_ship[item] = random.choice(self.planets)
        for item in self.items_to_ship:
            cfps = [spade.message.Message(to=str(a.communicator.jid),
                                          sender=str(self.communicator.jid),
                                          body=json.dumps(item.__dict__) + '|' +
                                               str(self.items_to_ship[item].x) + '|' + str(self.items_to_ship[item].y),
                                          thread='CNP-' + str(item),
                                          metadata={"performative": "call_for_proposal",
                                                    "turn": str(self.model.schedule.steps)}) for
                    a in self.ships if a.x == self.x and a.y == self.y]
            for c in cfps:
                self.send(c)
            self.start_times[item] = self.model.schedule.steps
            self.proposals[item] = []
            self.waiting_for_proposal.append(item)
        self.items_to_ship = dict()
        for i in [item for item in self.waiting_for_proposal if
                  self.model.schedule.steps - self.start_times[item] >= WAITING_TIME]:
            self.items_to_ship[i] = random.choice(self.planets)
            self.waiting_for_proposal.remove(i)
            del self.start_times[i]

    @staticmethod
    def portrayal_method():
        color = "blue"
        r = 6
        portrayal = {"Shape": "circle",
                     "Filled": "true",
                     "Layer": 1,
                     "Color": color,
                     "r": r}
        return portrayal

Le second type d'agents est le `Ship` qui est capable de se déplacer sur le `SpaceRoadNetwork`, et qui pour le moment affiche les messages qu'il a reçus:

In [7]:
class Ship(CommunicatingAgent):
    def __init__(self, name: string, planets: List, unique_id: int, model,
                 x, y, max_speed: float, environment):
        super().__init__(unique_id, model, name)
        self.preference_a = random.random()
        self.preference_b = random.random()
        self.preference_c = random.random()
        self.planets = planets
        self.x = x
        self.y = y
        self.max_speed = max_speed
        self.destination = None
        self.potential_destination = None
        self.waypoint = None
        self.previous_point = [p for p in self.planets if (p.x == self.x and p.y == self.y)][0]
        self.environment = environment
        self.item = None

    def move_to(self, dest, speed):
        movement = tuple(min(
            (speed * (dest.x - self.x) / np.linalg.norm((dest.x - self.x, dest.y - self.y)), speed * (dest.y - self.y) /
             np.linalg.norm((dest.x - self.x, dest.y - self.y))),
            (dest.x - self.x, dest.y - self.y), key=lambda p: np.linalg.norm(p)))
        self.x += movement[0]
        self.y += movement[1]

    def step(self):
        if self.waypoint is None and self.destination is not None:
            self.waypoint = nx.dijkstra_path(self.environment.current_graph,
                                             self.previous_point, self.destination, 'distance')[1]
        if self.waypoint is not None:
            self.move_to(self.waypoint, self.max_speed * self.environment.speed_modificator[
                (self.previous_point, self.waypoint)])
            self.item.x = self.x
            self.item.y = self.y
            if (self.x, self.y) == (self.waypoint.x, self.waypoint.y):
                self.previous_point = self.waypoint
                if self.waypoint == self.destination:
                    # deliver
                    self.waypoint = None
                    self.destination = None
                    self.model.items.remove(self.item)
                    self.model.computed_items_nb += 1
                    self.item = None
                else:
                    self.waypoint = nx.dijkstra_path(self.environment.current_graph,
                                                     self.previous_point, self.destination,
                                                     'distance')[1]  # 0 is current planet
        self.communicator.mutex.acquire()
        try:
            messages = [m for m in self.communicator.msg_box]
            self.communicator.msg_box = []
        finally:
            self.communicator.mutex.release()
        for m in messages:
            print(m)

    def utility(self, item):
        return item.a * self.preference_a + item.b * self.preference_b + item.c * self.preference_c

    def portrayal_method(self):
        portrayal = {"Shape": "arrowHead", "s": 1, "Filled": "true", "Color": "Red", "Layer": 2, 'x': self.x,
                     'y': self.y}
        if self.waypoint and not (self.waypoint.x == self.x and self.waypoint.y == self.y):
            if self.waypoint.y - self.y > 0:
                portrayal['angle'] = math.acos((self.waypoint.x - self.x) /
                                               np.linalg.norm((self.waypoint.x - self.x, self.waypoint.y - self.y)))
            else:
                portrayal['angle'] = 2 * math.pi - math.acos((self.waypoint.x - self.x) /
                                                             np.linalg.norm(
                                                                 (self.waypoint.x - self.x, self.waypoint.y - self.y)))
        else:
            portrayal['angle'] = 0
        return portrayal

Notez que les comportements de déplacement ont déjà été implémentés pour vous. De manière à raccourcir le TP, certains apects d'adaptation à l'environnement ont déjà été implémentés. C'est en particulier le cas de l'adaptation aux changements du `SpaceRoadNetwork` que vous devrez implémenter par la suite. C'est aussi cet agent qui livre les `Item` et ajoute donc une unité au `model_reporter` du modèle.

In [8]:
class PlanetDelivery(mesa.Model):

    def __init__(self, n_planets, n_ships):
        mesa.Model.__init__(self)
        self.space = mesa.space.ContinuousSpace(600, 600, False)
        self.schedule = RandomActivation(self)
        planets = [PlanetManager("planet-" + str(i), [], int(uuid.uuid1()), self,
                                 random.random() * 600, random.random() * 600)
                   for i in range(n_planets)]
        environment = SpaceRoadNetwork(planets, int(uuid.uuid1()), self)
        self.schedule.add(environment)
        ships = []
        for i in range(n_ships):
            starting_point = random.choice(planets)
            ship = Ship("ship-" + str(i), planets, int(uuid.uuid1()), self,
                        starting_point.x, starting_point.y, 60, environment)
            ships.append(ship)
            self.schedule.add(ship)
        for p in planets:
            p.planets = [planet for planet in planets if planet != p]
            p.ships = ships
            self.schedule.add(p)
        self.items = []
        self.computed_items_nb = 0
        self.datacollector = DataCollector(
            model_reporters={"items": lambda model: len(model.items),
                             "Delivered": lambda model: model.computed_items_nb
                             },
            agent_reporters={})

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        if self.schedule.steps >= 300:
            self.running = False

Le `Canvas` est légèrement plus complexe que celui du premier TP:

In [9]:
class ContinuousCanvas(VisualizationElement):
    local_includes = [
        "./js/simple_continuous_canvas.js",
    ]

    def __init__(self, canvas_height=500,
                 canvas_width=500, instantiate=True):
        VisualizationElement.__init__(self)
        self.canvas_height = canvas_height
        self.canvas_width = canvas_width
        self.identifier = "space-canvas"
        if instantiate:
            new_element = ("new Simple_Continuous_Module({}, {},'{}')".
                           format(self.canvas_width, self.canvas_height, self.identifier))
            self.js_code = "elements.push(" + new_element + ");"

    @staticmethod
    def portrayal_method(obj):
        return obj.portrayal_method()

    def render(self, model):
        representation = defaultdict(list)
        for obj in model.schedule.agents:
            portrayal = self.portrayal_method(obj)
            if portrayal:
                if isinstance(obj, SpaceRoadNetwork):
                    for p in portrayal:
                        representation[p["Layer"]].append(p)
                else:
                    portrayal["x"] = ((obj.x - model.space.x_min) /
                                      (model.space.x_max - model.space.x_min))
                    portrayal["y"] = ((obj.y - model.space.y_min) /
                                      (model.space.y_max - model.space.y_min))
                    representation[portrayal["Layer"]].append(portrayal)
        for obj in model.items:
            portrayal = self.portrayal_method(obj)
            portrayal["x"] = ((obj.x - model.space.x_min) /
                              (model.space.x_max - model.space.x_min))
            portrayal["y"] = ((obj.y - model.space.y_min) /
                              (model.space.y_max - model.space.y_min))
            representation[portrayal["Layer"]].append(portrayal)
        return representation

Si vous avez bien lancé prosody, alors le code suivant devrait vous permettre de lancer mesa, avec tous les agents immobiles, les biens créés et les messages envoyés.

In [10]:
def run_single_server():
    chart = ChartModule([{"Label": "items",
                          "Color": "Red"},
                         {"Label": "Delivered",
                          "Color": "Blue"}
                         ],
                        data_collector_name='datacollector')

    server = ModularServer(PlanetDelivery,
                           [ContinuousCanvas(), chart],
                           "PlanetDelivery",
                           {"n_planets": ModularVisualization.UserSettableParameter('slider',
                                                                                    "Number of planets",
                                                                                    10, 3, 20, 1),
                            "n_ships": ModularVisualization.UserSettableParameter('slider',
                                                                                  "Number of spaceships",
                                                                                  15, 3, 30, 1)})
    server.port = 8521
    server.launch()


if __name__ == "__main__":
    run_single_server()

Interface starting at http://127.0.0.1:8521
planet-2@localhost connected
planet-1@localhost connected
ship-0@localhost connected
planet-5@localhost connected
planet-3@localhost connected
planet-7@localhost connected
planet-4@localhost connected
planet-8@localhost connected
planet-6@localhost connected
ship-4@localhost connected
ship-1@localhost connected
ship-9@localhost connected
ship-3@localhost connected
ship-6@localhost connected
planet-9@localhost connected
planet-0@localhost connected
ship-14@localhost connected
ship-5@localhost connected
ship-7@localhost connected
ship-10@localhost connected
ship-13@localhost connected
ship-2@localhost connected
ship-11@localhost connected
ship-8@localhost connected
ship-12@localhost connected


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
planet-2@localhost connected
planet-0@localhost connected
planet-4@localhost connected
planet-8@localhost connected
planet-7@localhost connected
planet-9@localhost connected
ship-1@localhost connected
planet-6@localhost connected
ship-9@localhost connected
planet-5@localhost connected
planet-1@localhost connected
planet-3@localhost connected
ship-5@localhost connected
ship-7@localhost connected
ship-0@localhost connected
ship-3@localhost connected
ship-12@localhost connected
ship-11@localhost connected
ship-13@localhost connected
ship-14@localhost connected
ship-2@localhost connected
ship-10@localhost connected
ship-6@localhost connected
ship-8@localhost connected
ship-4@localhost connected
Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
planet-7@localhost connected
planet-4@localhost connected


No behaviour matched for message: <message to="ship-11@localhost" from="planet-9@localhost" thread="CNP-<__main__.Item object at 0x7f7a8c16c6a0>" metadata={'performative': 'call_for_proposal', 'turn': '2'}>
{"a": 0.8722355151571612, "b": 0.24508047979215353, "c": 0.4543684749576735, "x": 218.48107655415367, "y": 472.5299643835904, "uid": 145671688757985051364329642550894241685}|53.953057664356564|284.1130127017244
</message>


ship-2@localhost connected
ship-0@localhost connected
planet-5@localhost connected
planet-0@localhost connected
planet-8@localhost connected
ship-9@localhost connected
planet-6@localhost connected
planet-1@localhost connected
planet-9@localhost connected
ship-8@localhost connected
ship-6@localhost connected
ship-7@localhost connected
ship-13@localhost connected
sent: <message to="ship-6@localhost" from="planet-9@localhost" thread="CNP-<__main__.Item object at 0x7f7a8c16c6a0>" metadata={'performative': 'call_for_proposal', 'turn': '2'}>
{"a": 0.8722355151571612, "b": 0.24508047979215353, "c": 0.4543684749576735, "x": 218.48107655415367, "y": 472.5299643835904, "uid": 145671688757985051364329642550894241685}|53.953057664356564|284.1130127017244
</message>

planet-3@localhost connected
ship-10@localhost connected
ship-5@localhost connected
ship-1@localhost connected
ship-3@localhost connected
ship-12@localhost connected
planet-2@localhost connected
sent: <message to="ship-11@localhost" fr

planet-11@localhost connected
ship-22@localhost connected
planet-6@localhost connected
planet-7@localhost connected
ship-8@localhost connected
ship-7@localhost connected
ship-12@localhost connected
ship-5@localhost connected
ship-21@localhost connected
planet-14@localhost connected
ship-23@localhost connected
planet-10@localhost connected
ship-16@localhost connected
ship-19@localhost connected
planet-2@localhost connected
ship-20@localhost connected


In [1]:
import tornado, tornado.ioloop
tornado.ioloop.IOLoop.current().stop()

# Interaction et Organisation

Pour le moment, tout est statique car le seul message utilisé est un *call for proposal* envoyé par les `PlanetManager`, qui est uniquement reçu et affiché par le `Ship`. Dans cette partie, nous allons faire en sorte que le TP fonctionne, en particulier que les `Item` soient pris en charge par le `Ship` qui les préfère. Pour ce faire, nous allons mettre en place un *Contract Net Protocol* parcellaire, en laissant de côté tout ce qui se passe après l'`accept_proposal` ou le `reject_proposal`. Commencez par observer l'`AgentCommunicator`. Comme nous l'avons vu, cette classe est un agent de spade à laquelle sont ajoutées deux `Behaviour`, l'un qui reçoit, un `CyclicBehaviour` qui tourne toutes les 0.01s, et un qui sert à l'envoi, qui est un `OneShotBehaviour`.

Mettez en place un *Contract Net Protocol* de manière à ce qu'un `Ship` fasse une proposition lorsqu'il reçoit un *call for proposal* (CFP) avec pour corps: la version sérialisée du bien, et l'utilité calculée par le `Ship`. Il doit alors arrêter de répondre aux CFP afin d'éviter de s'engager sur deux `Item` différents, jusqu'à avoir une réponse du `PlanetManager`. Le *proposal* est envoyé au `PlanetManager` qui récolte tous les messages reçus, puis, une fois qu'un certain temps (ici `WAITING_TIME`) est écoulé, il vérifie s'il a des propositions. Si tel est le cas, il choisit le `Ship` ayant la meilleure utilité et lui envoie un `accept_proposal` et un `reject_proposal` aux autres. Sinon, il relance un `call_for_proposal` vers un autre `PlanetManager` et remet le bien dans `items_to_ship`. Lorsqu'un `Ship` reçoit un `accept_proposal`, il récupère l'item et le ramène.

In [ ]:
class PlanetManager(CommunicatingAgent):
    def __init__(self, name: string, ships: List, unique_id: int, model, x, y):
        super().__init__(unique_id, model, name)
        self.x = x
        self.y = y
        self.items_to_ship = {}
        self.waiting_for_proposal = []
        self.ships = ships
        self.start_times = dict()
        self.proposals = dict()
        self.planets = []

    def step(self):
        if random.random() < NEW_ITEM_PROBA:
            item = Item(self.x, self.y)
            self.model.items.append(item)
            self.items_to_ship[item] = random.choice(self.planets)
        for item in self.items_to_ship:
            cfps = [spade.message.Message(to=str(a.communicator.jid),
                                          sender=str(self.communicator.jid),
                                          body=json.dumps(item.__dict__) + '|' +
                                               str(self.items_to_ship[item].x) + '|' + str(self.items_to_ship[item].y),
                                          thread='CNP-' + str(item),
                                          metadata={"performative": "call_for_proposal",
                                                    "turn": str(self.model.schedule.steps)}) for
                    a in self.ships if a.x == self.x and a.y == self.y]
            for c in cfps:
                self.send(c)
            self.start_times[item] = self.model.schedule.steps
            self.proposals[item] = []
            self.waiting_for_proposal.append(item)
        self.items_to_ship = dict()
        for i in [item for item in self.waiting_for_proposal if
                  self.model.schedule.steps - self.start_times[item] >= WAITING_TIME]:
            self.items_to_ship[i] = random.choice(self.planets)
            self.waiting_for_proposal.remove(i)
            del self.start_times[i]

    @staticmethod
    def portrayal_method():
        color = "blue"
        r = 6
        portrayal = {"Shape": "circle",
                     "Filled": "true",
                     "Layer": 1,
                     "Color": color,
                     "r": r}
        return portrayal


class Ship(CommunicatingAgent):
    def __init__(self, name: string, planets: List, unique_id: int, model,
                 x, y, max_speed: float, environment):
        super().__init__(unique_id, model, name)
        self.preference_a = random.random()
        self.preference_b = random.random()
        self.preference_c = random.random()
        self.planets = planets
        self.x = x
        self.y = y
        self.max_speed = max_speed
        self.destination = None
        self.potential_destination = None
        self.waypoint = None
        self.previous_point = [p for p in self.planets if (p.x == self.x and p.y == self.y)][0]
        self.environment = environment
        self.item = None

    def move_to(self, dest, speed):
        movement = tuple(min(
            (speed * (dest.x - self.x) / np.linalg.norm((dest.x - self.x, dest.y - self.y)), speed * (dest.y - self.y) /
             np.linalg.norm((dest.x - self.x, dest.y - self.y))),
            (dest.x - self.x, dest.y - self.y), key=lambda p: np.linalg.norm(p)))
        self.x += movement[0]
        self.y += movement[1]

    def step(self):
        if self.waypoint is None and self.destination is not None:
            self.waypoint = nx.dijkstra_path(self.environment.current_graph,
                                             self.previous_point, self.destination, 'distance')[1]
        if self.waypoint is not None:
            self.move_to(self.waypoint, self.max_speed * self.environment.speed_modificator[
                (self.previous_point, self.waypoint)])
            self.item.x = self.x
            self.item.y = self.y
            if (self.x, self.y) == (self.waypoint.x, self.waypoint.y):
                self.previous_point = self.waypoint
                if self.waypoint == self.destination:
                    # deliver
                    self.waypoint = None
                    self.destination = None
                    self.model.items.remove(self.item)
                    self.model.computed_items_nb += 1
                    self.item = None
                else:
                    self.waypoint = nx.dijkstra_path(self.environment.current_graph,
                                                     self.previous_point, self.destination,
                                                     'distance')[1]  # 0 is current planet
        self.communicator.mutex.acquire()
        try:
            messages = [m for m in self.communicator.msg_box]
            self.communicator.msg_box = []
        finally:
            self.communicator.mutex.release()
        for m in messages:
            print(m)

    def utility(self, item):
        return item.a * self.preference_a + item.b * self.preference_b + item.c * self.preference_c

    def portrayal_method(self):
        portrayal = {"Shape": "arrowHead", "s": 1, "Filled": "true", "Color": "Red", "Layer": 2, 'x': self.x,
                     'y': self.y}
        if self.waypoint and not (self.waypoint.x == self.x and self.waypoint.y == self.y):
            if self.waypoint.y - self.y > 0:
                portrayal['angle'] = math.acos((self.waypoint.x - self.x) /
                                               np.linalg.norm((self.waypoint.x - self.x, self.waypoint.y - self.y)))
            else:
                portrayal['angle'] = 2 * math.pi - math.acos((self.waypoint.x - self.x) /
                                                             np.linalg.norm(
                                                                 (self.waypoint.x - self.x, self.waypoint.y - self.y)))
        else:
            portrayal['angle'] = 0
        return portrayal

**Question 3 – Lancez la simulation. Comment évoluent le rapport entre le nombre de biens présents dans le système et le nombre de biens livrés, en fonction du nombre de planètes et du nombre de vaisseaux. Ajoutez une image de la courbe.**

*Insérez votre réponse ici*

**Question 4&ast; – De quelle organisation s'agit-il ? Justifiez votre réponse**

*Insérez votre réponse ici*

# Environnement dynamique

La dernière étape du TP est de modifier l'environnement pour le rendre dynamique. En effet, comme sur nos routes, il arrive que les autoroutes de l'espace soient encombrées… par des météorites. Dans ce TP, on va considérer que ces autoroutes spatiales peuvent être dans trois états: en parfait état (leur état actuel, auquel cas les agents vont à leur vitesse maximale), encombrés (auquel cas il vont à mi-vitesse par rapport à leur vitesse maximale et s'affcihent en rouge) ou complètement impraticables (dans ce cas, les agents sont totalement immobilisés et la route ne doit pas être affichée). Notez que ces modifications sont déjà prévues pour l'agent `Ship`, inutile de le modifier. Modifiez le code pour prendre en compte ces changements. Un changement de statut intervient avec une probabilité `PROBA_ISSUE_ROAD`. S'il y a un changement de statut, la route a 50% de chance de passer dans chacun des états dans lequel il n'était pas:

In [ ]:
class SpaceRoadNetwork(Agent):
    def __init__(self, planets: List, unique_id: int, model: Model):
        super().__init__(unique_id, model)
        self.initial_graph = nx.Graph()
        self.current_graph = nx.Graph()
        self.speed_modificator = dict()
        for i in range(len(planets)):
            self.initial_graph.add_node(planets[i])
            for j in range(i):
                if random.random() < ROAD_BRANCHING_FACTOR:
                    distance = np.linalg.norm([planets[i].x - planets[j].x, planets[i].y - planets[j].y])
                    self.initial_graph.add_edge(planets[i], planets[j], distance=distance)
        # Reconnect graph of the roads between planets
        while len(list(nx.connected_components(self.initial_graph))) != 1:
            first_element = random.choice(tuple(list(nx.connected_components(self.initial_graph))[0]))
            second_element = random.choice(tuple(list(nx.connected_components(self.initial_graph))[1]))
            distance = np.linalg.norm([first_element.x - second_element.x, first_element.y - second_element.y])
            self.initial_graph.add_edge(first_element, second_element, distance=distance)
        for e in self.initial_graph.edges:
            self.current_graph.add_edge(e[0], e[1],
                                        distance=nx.get_edge_attributes(self.initial_graph, 'distance')[(e[0], e[1])])
            self.speed_modificator[e] = 1.0
            self.speed_modificator[(e[1], e[0])] = 1.0

    def step(self):
        pass  # À modifier

    def portrayal_method(self):  # À modifier
        portrayals = []
        for e in [edge for edge in self.current_graph.edges]:
            color = "green"
            portrayal = {"Shape": "line",
                         "width": 1,
                         "Layer": 1,
                         "Color": color,
                         "from_x": (tuple(e)[0].x - self.model.space.x_min) /
                                   (self.model.space.x_max - self.model.space.x_min),
                         "from_y": (tuple(e)[0].y - self.model.space.y_min) /
                                   (self.model.space.y_max - self.model.space.y_min),
                         "to_x": (tuple(e)[1].x - self.model.space.x_min) /
                                 (self.model.space.x_max - self.model.space.x_min),
                         "to_y": (tuple(e)[1].y - self.model.space.y_min) /
                                 (self.model.space.y_max - self.model.space.y_min)
                         }
            portrayals.append(portrayal)
        return portrayals

**Question 5 – Caractérisez l'environnement du système comme nous l'avons vu en cours. Justifiez.**

*Insérez votre réponse ici*

**Question 6 – Comment les agents s'adaptent-ils à ce changement d'environnement ? À quelle caractéristique fondamentale des agents cela correspond-il ?**

*Insérez votre réponse ici*

**Question 7 – Lancez la simulation. Quelle différence observez-vous avec la situation précédente ?**

*Insérez votre réponse ici*

**Question Bonus – Ajoutez le `ROAD_BRANCHING_FACTOR` aux paramètres à faire varier et étudier l'influence de ce facteur sur le nombre de biens livrés.**